In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import numpy as np


In [2]:
# Librerías
import pandas as pd
import numpy as np


movies_path = "ml-1m/movies.dat"
ratings_path = "ml-1m/ratings.dat"
users_path = "ml-1m/users.dat"


# Carga de los datos
users = pd.read_csv(users_path, sep="::", engine="python", names=["UserID", "Gender", "Age", "Occupation", "Zip-code"], encoding="latin-1")
movies = pd.read_csv(movies_path, sep="::", engine="python", names=["MovieID", "Title", "Genres"], encoding="latin-1")
ratings = pd.read_csv(ratings_path, sep="::", engine="python", names=["UserID", "MovieID", "Rating", "Timestamp"], encoding="latin-1")

# Mostrar primeras filas
ratings.head()



,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
from sklearn.preprocessing import LabelEncoder

user_enc = LabelEncoder()
movie_enc = LabelEncoder()

ratings['user'] = user_enc.fit_transform(ratings['UserID'])
ratings['movie'] = movie_enc.fit_transform(ratings['MovieID'])

num_users = ratings['user'].nunique()
num_movies = ratings['movie'].nunique()


In [4]:
from collections import defaultdict

max_seq_len = 10  # longitud fija de historial

user_histories = defaultdict(list)
sequence_data = []

# Ordenamos por timestamp
ratings = ratings.sort_values(by=['user', 'Timestamp'])

# Generar secuencias
for row in ratings.itertuples():
    u, m = row.user, row.movie
    hist = user_histories[u][-max_seq_len:]
    if len(hist) >= 1:  # solo si hay al menos un ítem en historial
        padded_hist = [0] * (max_seq_len - len(hist)) + hist  # pad left
        sequence_data.append((padded_hist, m))
    user_histories[u].append(m)


In [5]:
import torch
from torch.utils.data import Dataset

class GRU4RecDataset(Dataset):
    def __init__(self, sequence_data):
        self.sequences = [torch.tensor(s, dtype=torch.long) for s, _ in sequence_data]
        self.targets = [torch.tensor(t, dtype=torch.long) for _, t in sequence_data]

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.targets[idx]


In [6]:
from torch.utils.data import random_split, DataLoader

full_dataset = GRU4RecDataset(sequence_data)

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256)


In [7]:
import torch.nn as nn

class GRU4RecModel(nn.Module):
    def __init__(self, num_items, embedding_dim=64, hidden_dim=128, num_layers=1, dropout=0.2):
        super(GRU4RecModel, self).__init__()
        self.embedding = nn.Embedding(num_items, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.output_layer = nn.Linear(hidden_dim, num_items)

    def forward(self, input_seq):
        emb = self.embedding(input_seq)                         # (batch, seq_len, emb_dim)
        gru_out, _ = self.gru(emb)                              # (batch, seq_len, hidden_dim)
        last_hidden = gru_out[:, -1, :]                         # última salida (batch, hidden_dim)
        logits = self.output_layer(last_hidden)                # (batch, num_items)
        return logits


In [8]:
def train_gru4rec_model(model, train_loader, val_loader, epochs=10, lr=0.001, device='cpu'):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_train_loss = 0

        for input_seq, target in train_loader:
            input_seq, target = input_seq.to(device), target.to(device)

            optimizer.zero_grad()
            logits = model(input_seq)
            loss = criterion(logits, target)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        # Validación
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for input_seq, target in val_loader:
                input_seq, target = input_seq.to(device), target.to(device)
                logits = model(input_seq)
                loss = criterion(logits, target)
                val_loss += loss.item()

                preds = torch.argmax(logits, dim=1)
                correct += (preds == target).sum().item()
                total += target.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1}: Train Loss = {total_train_loss/len(train_loader):.4f}, "
              f"Val Loss = {val_loss/len(val_loader):.4f}, Val Acc = {acc:.4f}")


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
gru_model = GRU4RecModel(num_items=num_movies, embedding_dim=64, hidden_dim=128).to(device)

train_gru4rec_model(
    model=gru_model,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=10,
    lr=0.001,
    device=device
)


/home/hjasi/Documentos/DeepLearning/.venv/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


Epoch 1: Train Loss = 6.4826, Val Loss = 5.9732, Val Acc = 0.0384
Epoch 2: Train Loss = 5.7956, Val Loss = 5.7845, Val Acc = 0.0469
Epoch 3: Train Loss = 5.6105, Val Loss = 5.7196, Val Acc = 0.0494
Epoch 4: Train Loss = 5.5017, Val Loss = 5.6959, Val Acc = 0.0517
Epoch 5: Train Loss = 5.4248, Val Loss = 5.6919, Val Acc = 0.0517
Epoch 6: Train Loss = 5.3652, Val Loss = 5.6956, Val Acc = 0.0524
Epoch 7: Train Loss = 5.3162, Val Loss = 5.7047, Val Acc = 0.0528
Epoch 8: Train Loss = 5.2747, Val Loss = 5.7191, Val Acc = 0.0521
Epoch 9: Train Loss = 5.2388, Val Loss = 5.7325, Val Acc = 0.0527
Epoch 10: Train Loss = 5.2076, Val Loss = 5.7504, Val Acc = 0.0523
